In [42]:
from datetime import date, timedelta
from covid_registry import CovidRegistry
import pandas as pd
from pathlib import Path

In [3]:
grande_rio = [
    'Rio de Janeiro',
    'São Gonçalo',
    'Duque de Caxias',
    'Nova Iguaçu',
    'Niteroi',
    'Belford Roxo',
    'São João de Meriti',
    'Mage',
    'Itaborai',
    'Mesquita',
    'Nilópolis',
    'Maricá',
    'Queimados',
    'Itaguai',
    'Japeri',
    'Seropedica',
    'Guapimirim',
    'Paracambi',
    'Tanguá',
]

# Testes

As tabelas produzidas até agora apresentam problemas, com um número massivo de pontos repetidos e possivelmente pontos faltantes. Preciso determinar qual a origem do problema.

## 1. Sob quais condições os erros aparecem?

Os erros foram observados nos dados extraídos para o mês de janeiro relativos aos municípios da região metropolitana do Rio de Janeiro para ambos os gêneros e todos os locais de óbito. Esses parâmetros tem algum impacto na ocorrêcia do erro?

### 1.1 Limitando o intervalo de tempo

#### 1.1.1 Apenas um dia

Se realizar a consulta para apenas um dia, observo o erro?

In [4]:
name = 'tests/single_day/grande_rio.csv'
pkl = 'tests/single_day/remaining_tasks.pkl'

async with CovidRegistry() as reg:
    dates = pd.date_range(start='20200101', end='20200101')
    await reg.dump(dates, states=['RJ'], cities=grande_rio, file=name, pkl=pkl)

This operation will take 190 requests. Checkpoints will be created once every 200 requests.
#0001 -- 190/190 requests -- 0 failures -- Elapsed: 0:00:08.373151


Vamos, agora, validar o resultado. Alguns pontos para ter em mente: Apesar do número de requisições iniciais ser maior do que o tamanho do 'chunk', o fato de requisições mal sucedidas terem ocorrido, forçou a operação a se dar em 5 blocos, cada qual resultou em um dataframe que foi anexado ao arquivo final.

In [5]:
test_file = Path('tests/single_day/grande_rio.csv')

In [6]:
df = pd.read_csv(
    test_file,
    usecols=['Date', 'State', 'City', 'Age', 'Gender', 'Place of Death', 'Cause', '#'],
    parse_dates=[1]
)

In [7]:
df.head()

,Date,State,City,Age,Gender,Place of Death,Cause,#
0,2019-01-01,RJ,Rio de Janeiro,< 9,M,HOSPITAL,OUTRAS,1
1,2019-01-01,RJ,Rio de Janeiro,< 9,M,HOSPITAL,PNEUMONIA,1
2,2020-01-01,RJ,Rio de Janeiro,< 9,M,HOSPITAL,INSUFICIENCIA_RESPIRATORIA,1
3,2020-01-01,RJ,Rio de Janeiro,< 9,M,HOSPITAL,SEPTICEMIA,1
4,2019-01-01,RJ,Rio de Janeiro,20 - 29,M,HOSPITAL,OUTRAS,1


Vamos ordenar o dataframe e observar se temos repetições:

In [8]:
df.sort_values(by=['Date', 'State', 'City', 'Age', 'Gender', 'Place of Death', 'Cause'], axis=0, inplace=True)

In [9]:
df.head()

,Date,State,City,Age,Gender,Place of Death,Cause,#
248,2019-01-01,RJ,Belford Roxo,30 - 39,F,HOSPITAL,INSUFICIENCIA_RESPIRATORIA,1
246,2019-01-01,RJ,Belford Roxo,60 - 69,M,HOSPITAL,SEPTICEMIA,1
249,2019-01-01,RJ,Belford Roxo,80 - 89,F,HOSPITAL,OUTRAS,1
253,2019-01-01,RJ,Belford Roxo,90 - 99,F,DOMICILIO,OUTRAS,1
172,2019-01-01,RJ,Duque de Caxias,20 - 29,M,VIA_PUBLICA,OUTRAS,1


Visualmente, não parece haver repetições. Vamos tentar provar isso. O método `duplicated`, como usado a seguir, retorna uma série booleana onde toda linha que ocorre mais de uma vez é marcada como True. Se tivermos uma quantia não nula de verdadeiros nessa série, temos duplicatas:

In [10]:
df.duplicated(keep=False).value_counts()

False    333
dtype: int64

E se tivéssemos forçado o chunking?

In [11]:
name = 'tests/single_day/grande_rio.csv'
pkl = 'tests/single_day/remaining_tasks.pkl'

async with CovidRegistry() as reg:
    dates = pd.date_range(start='20200101', end='20200101')
    await reg.dump(dates, states=['RJ'], cities=grande_rio, file=name, pkl=pkl, checkpoint_once_in=50)

This operation will take 190 requests. Checkpoints will be created once every 50 requests.
#0001 -- 50/190 requests -- 0 failures -- Elapsed: 0:00:02.673167
#0002 -- 100/190 requests -- 0 failures -- Elapsed: 0:00:04.972636
#0003 -- 150/190 requests -- 0 failures -- Elapsed: 0:00:09.381964
#0004 -- 190/190 requests -- 0 failures -- Elapsed: 0:00:11.632529


Avaliando novamente os resultados:

In [12]:
df = pd.read_csv(
    test_file,
    usecols=['Date', 'State', 'City', 'Age', 'Gender', 'Place of Death', 'Cause', '#'],
    parse_dates=[1]
)

In [13]:
df.sort_values(by=['Date', 'State', 'City', 'Age', 'Gender', 'Place of Death', 'Cause'], axis=0, inplace=True)
df.head(12)

,Date,State,City,Age,Gender,Place of Death,Cause,#
248,2019-01-01,RJ,Belford Roxo,30 - 39,F,HOSPITAL,INSUFICIENCIA_RESPIRATORIA,1
246,2019-01-01,RJ,Belford Roxo,60 - 69,M,HOSPITAL,SEPTICEMIA,1
249,2019-01-01,RJ,Belford Roxo,80 - 89,F,HOSPITAL,OUTRAS,1
253,2019-01-01,RJ,Belford Roxo,90 - 99,F,DOMICILIO,OUTRAS,1
172,2019-01-01,RJ,Duque de Caxias,20 - 29,M,VIA_PUBLICA,OUTRAS,1
159,2019-01-01,RJ,Duque de Caxias,30 - 39,M,HOSPITAL,SEPTICEMIA,1
174,2019-01-01,RJ,Duque de Caxias,40 - 49,F,HOSPITAL,OUTRAS,1
160,2019-01-01,RJ,Duque de Caxias,40 - 49,M,HOSPITAL,OUTRAS,2
176,2019-01-01,RJ,Duque de Caxias,60 - 69,F,HOSPITAL,OUTRAS,1
177,2019-01-01,RJ,Duque de Caxias,60 - 69,F,HOSPITAL,SEPTICEMIA,1


In [14]:
df.duplicated(keep=False).value_counts()

False    333
dtype: int64